In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
!mkdir checkpoints

In [4]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':20,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE':4,
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
df = pd.read_csv('/kaggle/input/weatherss/weather_datas_final.csv')

In [7]:
df["label"] = df["label"].map({1:0, 2:1, 3:2})

In [8]:
df

,Unnamed: 0,sample_id,video_path,label
0,0,TRAIN_0000,/kaggle/input/dacon-car-crash/train/TRAIN_0000...,0
1,1,TRAIN_0001,/kaggle/input/dacon-car-crash/train/TRAIN_0001...,0
2,2,TRAIN_0002,/kaggle/input/dacon-car-crash/train/TRAIN_0002...,0
3,3,TRAIN_0003,/kaggle/input/dacon-car-crash/train/TRAIN_0003...,0
4,4,TRAIN_0004,/kaggle/input/dacon-car-crash/train/TRAIN_0004...,0
...,...,...,...,...
2693,2693,TRAIN_2693,/kaggle/input/dacon-car-crash/train/TRAIN_2693...,1
2694,2694,TRAIN_2694,/kaggle/input/dacon-car-crash/train/TRAIN_2694...,2
2695,2695,TRAIN_2695,/kaggle/input/dacon-car-crash/train/TRAIN_2695...,0
2696,2696,TRAIN_2696,/kaggle/input/dacon-car-crash/train/TRAIN_2696...,2


In [9]:
import random

# 라벨이 0인 데이터 중 1/4 비율만 남기고 제거
label_0_index = df.index[df['label'] == 0].tolist()
sample_size = int(len(label_0_index) / 6)

for i in random.sample(label_0_index, len(label_0_index) - sample_size):
    df = df.drop(i)

In [10]:
label_count = df['label'].value_counts()

print(label_count)

1    350
0    346
2    269
Name: label, dtype: int64


In [11]:
df

,Unnamed: 0,sample_id,video_path,label
6,6,TRAIN_0006,/kaggle/input/dacon-car-crash/train/TRAIN_0006...,1
7,7,TRAIN_0007,/kaggle/input/dacon-car-crash/train/TRAIN_0007...,0
8,8,TRAIN_0008,/kaggle/input/dacon-car-crash/train/TRAIN_0008...,0
9,9,TRAIN_0009,/kaggle/input/dacon-car-crash/train/TRAIN_0009...,1
10,10,TRAIN_0010,/kaggle/input/dacon-car-crash/train/TRAIN_0010...,0
...,...,...,...,...
2685,2685,TRAIN_2685,/kaggle/input/dacon-car-crash/train/TRAIN_2685...,0
2690,2690,TRAIN_2690,/kaggle/input/dacon-car-crash/train/TRAIN_2690...,2
2693,2693,TRAIN_2693,/kaggle/input/dacon-car-crash/train/TRAIN_2693...,1
2694,2694,TRAIN_2694,/kaggle/input/dacon-car-crash/train/TRAIN_2694...,2


In [12]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

In [13]:
len(train),len(val)

(772, 193)

In [14]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list, tfms):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.tfms = tfms
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            frames.append(img)
        frames = aug_video(frames, tfms=self.tfms)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)
    
def aug_video(vid, tfms):
    seed = random.randint(0,99999)
    aug_vid = []
    for x in vid:
        random.seed(seed)
        aug_vid.append((tfms(image = np.asarray(x)))['image'])
    return torch.from_numpy(np.stack(aug_vid))

tfms = A.Compose([
            A.Resize(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
            A.HorizontalFlip(p=0.5),
            A.Normalize()
            ], p=1)

In [15]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values,tfms=tfms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values, tfms=tfms)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [16]:
import torch.nn as nn
from torchvision.models.video import r3d_18

class ResNet3DModel(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet3DModel, self).__init__()
        self.feature_extract = r3d_18(pretrained=True)
        num_features = self.feature_extract.fc.in_features
        self.feature_extract.fc = nn.Identity()
        self.classifier = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [18]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [ ]:

model = ResNet3DModel()
model.to(device)
optimizer = torch.optim.SGD(params=model.parameters(), lr=CFG["LEARNING_RATE"], momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)



Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


  0%|          | 0.00/127M [00:00<?, ?B/s]

  0%|          | 0/193 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.94066] Val Loss : [0.67050] Val F1 : [0.67916]


  0%|          | 0/193 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

Epoch [2], Train Loss : [0.72520] Val Loss : [0.66295] Val F1 : [0.77711]


  0%|          | 0/193 [00:00<?, ?it/s]

In [73]:
torch.save(model.state_dict(),'/kaggle/working/resnet_best.pt')